In [ ]:
import os
import faiss
from RAGLibrary import Widgets, Define
from RAGLibrary import ExtractData as Extract
from RAGLibrary import MergeData as Merge

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
widgets_list = Widgets.create_name_form()

In [ ]:
config = Define.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
chunks_base = config["chunks_base"]
json_file_path = config["json_file_path"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

SEARCH_ENGINE = faiss.IndexFlatIP

In [ ]:
path = dcmt_path
Contents = LEVEL_VALUES[-1] if LEVEL_VALUES else None
print(Contents)

In [ ]:
# EXTRACT DATA
exceptions_path = "4.0. exceptions.json"
markers_path = "4.0. markers.json"
status_path = "4.0. status.json"
text_data = Extract.extract_data(dcmt_path, exceptions_path, markers_path, status_path)

In [ ]:
import json
extract_path = data_foler + "_extract.json"
with open(extract_path, "w", encoding="utf-8") as json_file:
    json.dump(text_data, json_file, indent=2, ensure_ascii=False)

print(f"Đã xuất dữ liệu ra {extract_path}")

In [ ]:
modified_path = data_foler + "_modified.json"
modified_data = Extract.process_json(extract_path)

with open(modified_path, "w", encoding="utf-8") as json_file:
    json.dump(modified_data, json_file, indent=2, ensure_ascii=False)

print(f"Đã tạo file mới với căn lề tại {modified_path}")

In [ ]:
merged_path = data_foler + "_merged.json"
merged_data = Merge.merge_lines(modified_path)
with open(merged_path, 'w', encoding='utf-8') as f:
    json.dump(merged_data, f, indent=2, ensure_ascii=False)

In [ ]:
"""
LOGIC MERGE:
Để 2 text được coi là 1 đoạn văn bản liên tục, có thể ghép lại với nhau. cần thỏa mãn:
Bắt buộc thỏa mãn điều kiện cơ bản, khi thỏa mãn điều kiện cơ bản, tiếp tục xét điều kiện thêm để ghép:
Điều kiện cơ bản {
    Text[i] MarketText = null
    and Text[i] LastWord.Width + Text[i] FontSize * 0.5 > Text[i-1] ExtraSpace
    and Same Format {
        Same CaseStyle or Style {
            Same CaseStyle {
                    Text[i] CaseStyle = Text[i-1] CaseStyle
                or  Text[i] CaseStyle = Text[i-1] LastWord.CaseStyle
                or  Text[i] FirstWord.CaseStyle = Text[i-1] CaseStyle
                or  Text[i] FirstWord.CaseStyle = Text[i-1] LastWord.CaseStyle
            }
            or Same Style {
                    Text[i] Style = Text[i-1] Style
                or  Text[i] Style = Text[i-1] LastWord.Style
                or  Text[i] FirstWord.Style = Text[i-1] Style
                or  Text[i] FirstWord.Style = Text[i-1] LastWord.Style
            }
        }
        and Same FontSize {
                | Text[i] FontSize - Text[i-1] Fontsize | < 0.3
            or  | Text[i] FontSize - Text[i-1] LastWord.Fontsize | < 0.3
            or  | Text[i] FirstWord.FontSize - Text[i-1] Fontsize | < 0.3
            or  | Text[i] FirstWord.FontSize - Text[i-1] LastWord.Fontsize | < 0.3
        }
    }
}

Điều kiện thêm {
        (Text[i-1] BracketStatus = Open || Both) && [(Text[i] BracketStatus = Close || Both) || (Text[i+1] BracketStatus = Close || Both)]
    or  Margintop[i] < Margintop[i-1] + 0.3 && Margintop[i] < Margintop[i+1] + 0.3
}






Căn 2 lề: L <= 0, R <= 0, M = 0
Căn giữa: L > 0,  R > 0,  M = 0
Căn trái: L = ?,  R = ?,  M < 0
Căn phải: L = ?,  R = ?,  M > 0

Điều kiện cơ bản: 
  MarkerText[2] = null
& FontSize[2] = Fontsize[1]
& CaseStyle[2] = CaseStyle[1] /or/ Style[2] = Style[1]
& Top[2] < Top[1] + 0.3
& Top[2] < 2 * LineHeight[2]

Giữa - Giữa: Any
2 Lề - 2 Lề: Any
Phải - Phải: Any
Any  - Trái: ExtraSpace[1] < FirstWord[2] - Weight*1.3
Any  - Phải: MarginLeft[1] < FirstWord[2] - Weight*1.3
else: Không Merge


* Any: Cấu trúc hoặc đặc điểm bất kì đều được chấp nhận
* [2] là dòng sau, [1] là dòng trước

Lưu ý về việc so sánh bằng của các thuộc tính:
Same CaseStyle {
        Text[i] CaseStyle = Text[i-1] CaseStyle
    or  Text[i] CaseStyle = Text[i-1] LastWord.CaseStyle
    or  Text[i] FirstWord.CaseStyle = Text[i-1] CaseStyle
    or  Text[i] FirstWord.CaseStyle = Text[i-1] LastWord.CaseStyle
}
Same Style {
        Text[i] Style = Text[i-1] Style
    or  Text[i] Style = Text[i-1] LastWord.Style
    or  Text[i] FirstWord.Style = Text[i-1] Style
    or  Text[i] FirstWord.Style = Text[i-1] LastWord.Style
}
Same FontSize {
        | Text[i] FontSize - Text[i-1] Fontsize | < 0.3
    or  | Text[i] FontSize - Text[i-1] LastWord.Fontsize | < 0.3
    or  | Text[i] FirstWord.FontSize - Text[i-1] Fontsize | < 0.3
    or  | Text[i] FirstWord.FontSize - Text[i-1] LastWord.Fontsize | < 0.3
}










Final: 


Điều kiện cơ bản: 
  MarkerText[2] = null
& FontSize[2] = Fontsize[1]
& CaseStyle[2] = CaseStyle[1] /or/ Style[2] = Style[1]
& Top[2] < Top[1] + 0.3
& Top[2] < 2 * LineHeight[2]

Giữa - Giữa: Any
2 Lề - 2 Lề: Any
Phải - Phải: Any
Any  - Trái: ExtraSpace[1] < FirstWord[2] - Weight*1.3
Any  - Phải: MarginLeft[1] < FirstWord[2] - Weight*1.3
else: Không Merge


* Any: Cấu trúc hoặc đặc điểm bất kì đều được chấp nhận
* [2] là dòng sau, [1] là dòng trước

Lưu ý về việc so sánh bằng của các thuộc tính:
Same CaseStyle {
        Text[i] CaseStyle = Text[i-1] CaseStyle
    or  Text[i] CaseStyle = Text[i-1] LastWord.CaseStyle
    or  Text[i] FirstWord.CaseStyle = Text[i-1] CaseStyle
    or  Text[i] FirstWord.CaseStyle = Text[i-1] LastWord.CaseStyle
}
Same Style {
        Text[i] Style = Text[i-1] Style
    or  Text[i] Style = Text[i-1] LastWord.Style
    or  Text[i] FirstWord.Style = Text[i-1] Style
    or  Text[i] FirstWord.Style = Text[i-1] LastWord.Style
}
Same FontSize {
        | Text[i] FontSize - Text[i-1] Fontsize | < 0.3
    or  | Text[i] FontSize - Text[i-1] LastWord.Fontsize | < 0.3
    or  | Text[i] FirstWord.FontSize - Text[i-1] Fontsize | < 0.3
    or  | Text[i] FirstWord.FontSize - Text[i-1] LastWord.Fontsize | < 0.3
}

đây là các điều kiện để kiểm tra xem 2 đoạn có thể được ghép với nhau không.

code Python để thực hiện merge lại file vừa xuất 
modified_path là biến tôi đã đặt trước cho file vừa xuất, không đặt lại để tránh lãng phí tài nguyên.

cách merge là duyệt qua các đoạn có thể merge trước rồi merge luôn, ví dụ: 

para 1 : không xét
para 2: có thể gộp với para 1
para 3: có thể gộp với para 2
para 4: không thể gộp với para 3
tiến hành gộp para 1 - 2 - 3
tiếp tục xét 5, 6, ...
para 5
para 6

Đoạn văn được merge sẽ có các thuộc tính sau của đối tượng sau.

    {
      "Line": Auto number,
      "Text": Text sau khi đã gộp ngăn cách bằng space,
      "MarkerText": của para đầu tiên trong nhóm,
      "MarkerFormat": của para đầu tiên trong nhóm,
      "CaseStyle": Case của "Text" sau khi merge,
      "Style": 3 chữ số, mỗi chữ số là min của chữ số tương ứng của các đoạn đã merge,
      "FirstWord": Lấy của para đầu tiên trong nhóm,
      "LastWord": Lấy của para cuối cùng trong nhóm,
      "FontSize": Trung bình cộng các para trong nhóm,
      "Left": Min,
      "Top": của para đầu tiên,
      "Right": Min,
      "Mid": Trung bình cộng,
      "Align": "Tính lại sau khi biết Các thuộc tính trên"
    },


    

"""